In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
path = 'D:/work/pythonPractice/big projects/Blitzkrieg'
df = pd.read_csv(path + '/Original/Tree_case_data.csv')
df.shape